In [ ]:
!pip install torch -q
!pip install transformers -q
!pip install numpy -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain-chroma -q
!pip install sentence_transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Install

In [ ]:
pip install "numpy<2" --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have 

In [ ]:
import os
from google.colab import userdata

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch

model_id = "mistralai/Mistral-7B-v0.1"

huggingface_api_token = userdata.get('HUGGINGFACE_API_KEY')

tokenizer = AutoTokenizer.from_pretrained(model_id, token=huggingface_api_token)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, token=huggingface_api_token)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    trust_remote_code=True,
    device_map="auto",
)


llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
  model_name="sentence-transformers/all-mpnet-base-v2"
)

In [ ]:
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

In [ ]:
!pip install pypdf -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 9.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/FAS HANDBOOK.pdf")

docs = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

splits = text_splitter.split_documents(docs)

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer this question using the provided context only.

{question}

Context:
{context}

Answer:
"""
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [ ]:
response = chain.invoke("What is South Eastern University of Sri Lanka?")
print(response)

Human: 
Answer this question using the provided context only.

What is South Eastern University of Sri Lanka?

Context:
[Document(id='7dfed619-0ead-470f-8f67-ba8b72ffa290', metadata={'author': 'Dee', 'moddate': '2023-08-22T16:17:29+05:30', 'source': '/content/FAS HANDBOOK.pdf', 'producer': 'Foxit PDF Creator Version 9.0.0.1017', 'title': '', 'keywords': '', 'company': 'SEUSL', 'creator': 'Foxit Software Inc.', 'creationdate': '2023-08-22T16:17:29+05:30', 'page_label': '4', 'page': 3, 'total_pages': 99, 'subject': ''}, page_content='Page | 4  \n \n \n1 INTRODUCTION \n1.1 South Eastern University of Sri Lanka \nThe South Eastern University of Sri Lanka (SEUSL) was first established as the \nSouth Eastern University College of Sri Lanka and commenced to function from \n27th July 1995.  It was then upgraded to the status of a fully-fledged university, \nSEUSL, from 15th May 1996. There are six faculties in SEUSL. The Faculties of'), Document(id='d3cb3822-be81-48a0-958a-2ff4ea889940', metad